In [ ]:
library(CAGEfightR)
library(GenomicFeatures)
library(diptest)
setwd("./")

In [7]:
# bw_plus <- BigWigFileList(c("./Flower/PI46.Flower.plus.bw","./Leaf/PI46.Leaf.plus.bw",
#                             "./Nodule/PI46.Nodule.plus.bw","./Pod/PI46.Pod.plus.bw",
#                             "./Root/PI46.Root.plus.bw","./Seed/PI46.Seed.plus.bw",
#                             "./Shoot/PI46.Shoot.plus.bw","./Stemtip/PI46.Stemtip.plus.bw"))

# bw_minus <- BigWigFileList(c("./Flower/PI46.Flower.minus.bw","./Leaf/PI46.Leaf.minus.bw",
#                             "./Nodule/PI46.Nodule.minus.bw","./Pod/PI46.Pod.minus.bw",
#                             "./Root/PI46.Root.minus.bw","./Seed/PI46.Seed.minus.bw",
#                             "./Shoot/PI46.Shoot.minus.bw","./Stemtip/PI46.Stemtip.minus.bw"))

# names(bw_plus) <- c("Flower","Leaf",
#                     "Nodule","Pod",
#                     "Root","Seed",
#                     "Shoot","Stemtip")

# names(bw_minus) <-  c("Flower","Leaf",
#                       "Nodule","Pod",
#                       "Root","Seed",
#                       "Shoot","Stemtip")

In [2]:
n = "Leaf"

In [3]:
bw_plus = BigWigFileList(paste(n,"/PI46.",n,".plus.bw",sep = ""))
bw_minus = BigWigFileList(paste(n,"/PI46.",n,".minus.bw",sep = ""))
names(bw_plus) = c(n)
names(bw_minus) = c(n)


In [ ]:
bw_plus[[1]]

In [ ]:
Gm <- seqinfo(bw_plus[[1]])
Gm

In [ ]:
CTSSs <- quantifyCTSSs(plusStrand=bw_plus,
                       minusStrand=bw_minus,
                       genome=Gm)
TSSs <- quickTSSs(CTSSs)

In [ ]:

txdb <- makeTxDbFromGFF(file="/Data6/wanjie/other_data_archives/STRIPE/PI46/genome/PI46.final.gff3",
                         #file="Gmax_merged.gff3",
                        format="gff3",
                        dataSource="gff for Gm",
                       organism="Glycine max")

In [ ]:
# Annotate both TSSs and enhancers
CTSSs <- assignTxType(CTSSs, txModels=txdb,
                     outputColumn="txType")
#####
CTSSs <- assignTxID(CTSSs,
                   txModels=txdb,
                   outputColumn="txID")

In [ ]:
CTSSs <- calcTPM(CTSSs,
                 inputAssay="counts",
                 outputAssay="TPM",
                 outputColumn="subsetTags")

In [ ]:
CTSSs

In [12]:
###export a table
result <- data.frame(as.data.frame(CTSSs@rowRanges),
                     as.data.frame(as.matrix(CTSSs@assays@data@listData$counts)),
                    as.data.frame(as.matrix(CTSSs@assays@data@listData$TPM)))



In [13]:
write.table(result,
            file= paste("./",n,"/CTSS_raw.txt",sep=""),
            col.names=T,row.names=F,
            sep="\t",quote=F)

In [ ]:
CTSSs <- calcPooled(CTSSs, inputAssay="TPM")
rowRanges(CTSSs)

In [ ]:
CTSSs <- calcSupport(CTSSs,inputAssay="TPM", 
                            outputColumn="support", 
                            unexpressed=1)
table(rowRanges(CTSSs)$support)

In [ ]:
## ----subset, tidy=FALSE-------------------------------------------------------
supportedCTSSs <- subset(CTSSs, support > 0)
#supportedCTSSs <- calcTPM(supportedCTSSs, totalTags="totalTags")
supportedCTSSs <- calcPooled(supportedCTSSs)

In [17]:
result <-data.frame(as.data.frame(supportedCTSSs@rowRanges),
                    as.data.frame(as.matrix(assay(supportedCTSSs,"counts"))),
                    as.data.frame(as.matrix(assay(supportedCTSSs,"TPM"))))


In [18]:
write.table(result,
            file=paste("./",n,"/supportTSSs.txt",sep=""),
            col.names=T,row.names=F,
            sep="\t",quote=F)

In [ ]:
#change !!!
prefiltered_TCs <- clusterUnidirectionally(supportedCTSSs, 
                                           pooledCutoff=0, 
                                           mergeDist=20)
prefiltered_TCs

In [ ]:
#potential change the sample number.
sup_TSSs <- quantifyClusters(supportedCTSSs,
                                      clusters=prefiltered_TCs,
                                      inputAssay="counts")
sup_TSSs

In [ ]:
## ----supportOnTPM, tidy=FALSE-------------------------------------------------
# Calculate TPM using pre-calculated total tags:
Unidirectional <- calcTPM(sup_TSSs,
                          inputAssay="counts", 
                          outputAssay="TPM", 
                          outputColumn="subsetTags")
Unidirectional

In [ ]:
# Only TSSs expressed at more than 1 TPM in more than 2 samples
#Unidirectional <- subsetBySupport(Unidirectional,
#                                  inputAssay="TPM",
#                                  unexpressed=0,
#                                  minSamples=2)
#TSSs expressed at more than 0 TPM as least in 1 samples
Unidirectional <- subsetBySupport(Unidirectional,
                                     inputAssay="TPM",
                                     unexpressed=1,
                                     minSamples=0)
Unidirectional

In [ ]:
## ----assignTxID, tidy=FALSE---------------------------------------------------
Unidirectional <- assignTxID(Unidirectional,
                            txModels=txdb,
                            outputColumn="txID")

In [ ]:
## ----multipleTxs, tidy=FALSE--------------------------------------------------
rowRanges(Unidirectional)[5:6]
#export(rowRanges(Unidirectional),"TC.bed","BED")

In [ ]:
## ----assignTxType, tidy=FALSE-------------------------------------------------
Unidirectional <- assignTxType(Unidirectional,
                              txModels=txdb,
                              outputColumn="txType")

In [ ]:
## ----swappedTxType, tidy=FALSE------------------------------------------------
Unidirectional <- assignTxType(Unidirectional,
                              txModels=txdb,
                              outputColumn="peakTxType",
                              swap="thick")

In [ ]:
# Calculate shape
Unidirectional <- calcShape(Unidirectional,
                           pooled=supportedCTSSs,
                           outputColumn = "IQR",
                           shapeFunction = shapeIQR,
                           lower=0.25, upper=0.75)
Unidirectional
Unidirectional <- calcShape(Unidirectional,
                        pooled=supportedCTSSs,
                        outputColumn = "Entropy",
                        shapeFunction = shapeEntropy
                        )
Unidirectional

In [ ]:
## ----histIQR, tidy=FALSE------------------------------------------------------
hist(rowRanges(Unidirectional)$IQR,
     breaks=max(rowRanges(Unidirectional)$IQR),
     xlim=c(0,50), 
     xlab = "IQR", 
     col="red")

hist(rowRanges(Unidirectional)$Entropy,
     breaks=max(rowRanges(Unidirectional)$Entropy),
     xlim=c(0,20), 
     xlab = "Entropy", 
     col="red")

In [29]:
## ----customShape, tidy=FALSE--------------------------------------------------
# Write a function that quantifies the lean of a TSS

TSRshapeIndex <- function(x) {
  x <- as.vector(x)
  tagcount <- sum(x)  
  SI <- suppressWarnings(2+sum(ifelse(x > 0, (x/tagcount) * log2(x/tagcount), 0)))  
  return(SI)
}

TSRshapeIndex <- function(x) {
  x <- as.vector(x)
  tagcount <- sum(x) 
  SI <- suppressWarnings(2+sum(ifelse(x > 0, (x/tagcount) * log2(x/tagcount), 0))) 
  return(SI)
}

TSRmshapeIndex <- function(x) {
  x <- as.vector(x)
  tagcount <- sum(x)
  tsscount <- length(x)
  mSI <- suppressWarnings(1+sum(ifelse((tsscount != 1 & x != 0), (x/tagcount) * log((x/tagcount))/log(tsscount), 0)))
  return(mSI)
}

TSRshapiro <- function(x) {
  x <- as.vector(x)
  x[1]<-x[1]+0.0000001 
  shapiro <-suppressWarnings(ifelse(length(x) >=3, shapiro.test(x)$p.value,0))
  return(shapiro)
}


TSRdip <- function(x) {
  x <- as.vector(x)
  dip <-suppressWarnings(ifelse(length(x) >=3, dip.test(x)$p.value,1))
  return(dip)
}

In [ ]:
Unidirectional <- calcShape(Unidirectional,
                    pooled=supportedCTSSs,
                    outputColumn = "SI",
                    shapeFunction = TSRshapeIndex)
Unidirectional

In [ ]:
Unidirectional <- calcShape(Unidirectional,
                        pooled=supportedCTSSs,
                        outputColumn = "mSI",
                        shapeFunction = TSRmshapeIndex)
Unidirectional

In [ ]:
Unidirectional <- calcShape(Unidirectional,
                        pooled=supportedCTSSs,
                        outputColumn = "shapiro",
                        shapeFunction = TSRshapiro)
Unidirectional

In [ ]:
Unidirectional <- calcShape(Unidirectional,
                        pooled=supportedCTSSs,
                        outputColumn = "Dip",
                        shapeFunction = TSRdip)
Unidirectional

In [ ]:
hist(rowRanges(Unidirectional)$SI,
    # breaks=max(rowRanges(Unidirectional)$SI),
     #  xlim=c(0,20), 
     xlab = "SI", 
     col="red")

In [35]:
result <- data.frame(as.data.frame(Unidirectional@rowRanges),
                   as.data.frame(as.matrix(Unidirectional@assays@data@listData$counts)),
                   as.data.frame(as.matrix(Unidirectional@assays@data@listData$TPM)))



In [36]:
write.table(as.data.frame(result),
            file=paste("./",n,"/Unidirection_result.txt",sep = ""),
            col.names=T,row.names=F,
            sep="\t",quote=F)

In [ ]:
BCs <- clusterBidirectionally(supportedCTSSs, balanceThreshold=0)#0.95

In [ ]:
BCs <- calcBidirectionality(BCs, samples=supportedCTSSs)

BCs <- quantifyClusters(supportedCTSSs,
                        clusters=BCs,
                        inputAssay="counts")

BCs <- subsetBySupport(BCs,
                       inputAssay="counts",
                       unexpressed=0,
                       minSamples=0)

In [ ]:
Bidirectional <- calcTPM(BCs,
                         inputAssay="counts", 
                         outputAssay="TPM", 
                         outputColumn="subsetTags")

Bidirectional <- subsetBySupport(Bidirectional,
                                 inputAssay="TPM",
                                 unexpressed=1,
                                 minSamples=0)

Bidirectional <- assignTxType(Bidirectional,
                              txModels=txdb,
                              outputColumn="txType")

Bidirectional <- assignTxID(Bidirectional,
                            txModels=txdb,
                            outputColumn="txID")

In [40]:
# Only keep intronic and intergenic enhancers
#Bidirectional <- subset(Bidirectional,
#                               txType %in% c("intron", "intergenic"))
result<-data.frame(as.data.frame(Bidirectional@rowRanges),
                   as.data.frame(as.matrix(Bidirectional@assays@data@listData$counts)),
                   as.data.frame(as.matrix(Bidirectional@assays@data@listData$TPM)))



In [41]:
write.table(as.data.frame(result),
            file = paste("./",n,"/Bidirection_result.txt",sep=""),
            col.names=T,row.names=F,
            sep="\t",quote=F)
